# Classiflow End-to-End Reproducible Notebook
## MR Spectroscopy MB Molecular Group Classification (Meta, Sklearn)

## 0. Title + Reviewer-Friendly Overview

Classiflow provides reproducible, artifact-traceable ML workflows for technical validation and deployment gating. This notebook demonstrates a **technical-validation-only** workflow for medulloblastoma (MB) molecular group classification from MR spectroscopy features.

Scope of this notebook:
- Train/validate with nested CV using Classiflow `project run-technical`.
- Summarize technical artifacts (metrics, inner/outer CV outputs, lineage, resolved config).
- Evaluate the **Research / Exploratory Gate** on technical results.

Important study note:
- This dataset setup has **no independent test manifest** configured.
- Therefore, this notebook reports **technical validation status only** and explicitly does not claim independent-test promotion readiness.


## 1. Environment & Imports


In [1]:
import os
# os.environ.setdefault("MPLBACKEND", "Agg")

import hashlib
import json
import platform
import random
import shutil
import subprocess
import sys
from datetime import datetime
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yaml

from classiflow import __version__ as classiflow_version
from classiflow.projects.project_models import ProjectConfig, ThresholdsConfig
from classiflow.projects.promotion_templates import get_promotion_gate_template
from classiflow.projects.promotion import resolve_metric

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

print(f"Timestamp: {datetime.now().isoformat()}")
print(f"Python: {sys.version.split()[0]}")
print(f"Platform: {platform.platform()}")
print(f"Classiflow: {classiflow_version}")
print(f"NumPy: {np.__version__}")
print(f"Pandas: {pd.__version__}")
print(f"Matplotlib: {matplotlib.__version__}")

try:
    import sklearn
    print(f"scikit-learn: {sklearn.__version__}")
except Exception as exc:
    print(f"scikit-learn version unavailable: {exc}")

try:
    git_hash = subprocess.run(["git", "rev-parse", "HEAD"], capture_output=True, text=True, check=False).stdout.strip()
    print(f"Git commit: {git_hash if git_hash else 'unavailable'}")
except Exception:
    print("Git commit: unavailable")


Timestamp: 2026-02-09T12:38:34.347049
Python: 3.11.10
Platform: macOS-15.7.3-x86_64-i386-64bit
Classiflow: 0.1.0
NumPy: 1.26.4
Pandas: 2.2.3
Matplotlib: 3.9.2
scikit-learn: 1.5.2
Git commit: 813b91f311739b7591817da6b390d18ff53653c6


## 2. Load Data


In [2]:
TRAIN_CSV = Path("../data/MBmerged-z-scores_MLready_correction.csv")
LABEL_COL = "MOLECULAR"

if not TRAIN_CSV.exists():
    raise FileNotFoundError(f"Training dataset not found: {TRAIN_CSV}")

train_df = pd.read_csv(TRAIN_CSV)

if LABEL_COL not in train_df.columns:
    raise ValueError(f"Missing required label column: {LABEL_COL}")

feature_cols = [c for c in train_df.columns if c != LABEL_COL]
if not feature_cols:
    raise ValueError("No feature columns found after excluding label column")

print(f"Train shape: {train_df.shape}")
print(f"Feature columns: {len(feature_cols)}")
print(f"Classes: {sorted(train_df[LABEL_COL].dropna().unique().tolist())}")

missing_summary = pd.DataFrame({
    "missing_pct": train_df.isna().mean().mul(100).round(3)
}).sort_values("missing_pct", ascending=False)

print("Top missingness rows:")
display(missing_summary.head(10))

leakage_tokens = ("label", "class", "target", "outcome", "y_")
suspicious = [
    c for c in train_df.columns
    if c != LABEL_COL and any(tok in c.lower() for tok in leakage_tokens)
]
assert not suspicious, (
    "Potential label leakage columns detected (heuristic). "
    f"Please review/remove before training: {suspicious}"
)
print("Label leakage heuristic check: PASS")


Train shape: (94, 28)
Feature columns: 27
Classes: ['G3', 'G4', 'SHH', 'WNT']
Top missingness rows:


,missing_pct
MOLECULAR,0.0
Ala_conc,0.0
tIns_conc,0.0
tCr_conc.tCho_conc,0.0
tCr_conc.Glx_conc,0.0
tCr_conc,0.0
tCho_conc,0.0
Tau_conc.tCho_conc,0.0
Tau_conc,0.0
Scyllo_conc,0.0


Label leakage heuristic check: PASS


## 3. Data Distributions (Reviewer-Facing)


In [3]:
class_counts = train_df[LABEL_COL].value_counts().sort_index()
class_props = (class_counts / class_counts.sum() * 100).round(2)

summary_table = pd.DataFrame({
    "count": class_counts,
    "percent": class_props,
})
display(summary_table)

fig, ax = plt.subplots(figsize=(10, 4), constrained_layout=True)
class_counts.plot(kind="bar", ax=ax)
ax.set_title("Train Class Counts")
ax.set_xlabel("Class")
ax.set_ylabel("Count")
ax.tick_params(axis="x", rotation=45)
plt.show()



,count,percent
MOLECULAR,,
G3,22,23.40
G4,35,37.23
SHH,26,27.66
WNT,11,11.70


/var/folders/qw/wt3crg911d32djy9xk8j0q8r0000gn/T/ipykernel_74743/4029106374.py:16: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## 4. Create / Display Project Configuration


In [4]:
FAST_MODE = True

if FAST_MODE:
    outer_folds = 3
    inner_folds = 3
    repeats = 1
    candidates = ["logistic_regression", "svm", "random_forest"]
else:
    outer_folds = 5
    inner_folds = 5
    repeats = 2
    candidates = ["logistic_regression", "svm", "random_forest", "gradient_boost"]

OUTPUT_ROOT = Path("runs/mrs_mb_meta_sklearn_technical")
PROJECT_DIR = OUTPUT_ROOT / "project"
PROJECT_DIR.mkdir(parents=True, exist_ok=True)
(PROJECT_DIR / "registry").mkdir(parents=True, exist_ok=True)
(PROJECT_DIR / "runs").mkdir(parents=True, exist_ok=True)
(PROJECT_DIR / "promotion").mkdir(parents=True, exist_ok=True)

project_yaml = PROJECT_DIR / "project.yaml"
thresholds_yaml = PROJECT_DIR / "registry" / "thresholds.yaml"
labels_yaml = PROJECT_DIR / "registry" / "labels.yaml"
features_yaml = PROJECT_DIR / "registry" / "features.yaml"

project_payload = {
    "project": {
        "id": "MRS_MB_META_TECH",
        "name": "MRS MB Meta Technical",
        "description": "Reviewer notebook technical-validation-only run",
        "owner": "local",
    },
    "data": {
        "train": {"manifest": str(TRAIN_CSV.resolve())},
        "test": None,
    },
    "key_columns": {
        "sample_id": None,
        "patient_id": None,
        "label": LABEL_COL,
        "slide_id": None,
        "specimen_id": None,
    },
    "task": {
        "mode": "meta",
        "patient_stratified": False,
        "hierarchy_path": None,
    },
    "validation": {
        "nested_cv": {
            "outer_folds": outer_folds,
            "inner_folds": inner_folds,
            "repeats": repeats,
            "seed": SEED,
        }
    },
    "models": {
        "candidates": candidates,
        "selection_metric": "f1",
        "selection_direction": "max",
    },
    "imbalance": {
        "smote": {
            "enabled": False,
            "compare": False,
        }
    },
    "metrics": {
        "primary": ["f1", "balanced_accuracy"],
        "averaging": "macro",
        "include_confidence_intervals": False,
    },
    "calibration": {
        "calibrate_meta": True,
        "method": "sigmoid",
        "cv": 3,
        "bins": 10,
        "isotonic_min_samples": 100,
    },
    "final_model": {
        "sampler": None,
        "sanity_min_std": 0.02,
        "sanity_max_mean_deviation": 0.15,
        "train_from_scratch": True,
        "verify_dataset_hash": True,
    },
    "bundle": {
        "name": "model_bundle",
        "include_preprocessing": True,
        "format": "zip",
    },
    "execution": {
        "engine": "sklearn",
    },
}

thresholds_payload = {
    "promotion_gate_template": "research_exploratory",
    "technical_validation": {"required": {}, "safety": {}, "stability": None},
    "independent_test": {"required": {}, "safety": {}, "stability": None},
    "promotion": {"calibration": {"brier_max": None, "ece_max": None}},
}

cfg = ProjectConfig.model_validate(project_payload)
cfg.save(project_yaml)
ThresholdsConfig.model_validate(thresholds_payload).save(thresholds_yaml)
labels_yaml.write_text("labels: {}\n", encoding="utf-8")
features_yaml.write_text("features: {}\n", encoding="utf-8")

print("Wrote project config:", project_yaml)
print("Wrote thresholds config:", thresholds_yaml)
print("\nRequested config (YAML):")
print(yaml.safe_dump(project_payload, sort_keys=False))

print("Resolved/effective config:")
print(yaml.safe_dump(ProjectConfig.load(project_yaml).to_yaml_dict(), sort_keys=False))


Wrote project config: runs/mrs_mb_meta_sklearn_technical/project/project.yaml
Wrote thresholds config: runs/mrs_mb_meta_sklearn_technical/project/registry/thresholds.yaml

Requested config (YAML):
project:
  id: MRS_MB_META_TECH
  name: MRS MB Meta Technical
  description: Reviewer notebook technical-validation-only run
  owner: local
data:
  train:
    manifest: /Users/alex/Documents/project-MLSubtype/data/MBmerged-z-scores_MLready_correction.csv
  test: null
key_columns:
  sample_id: null
  patient_id: null
  label: MOLECULAR
  slide_id: null
  specimen_id: null
task:
  mode: meta
  patient_stratified: false
  hierarchy_path: null
validation:
  nested_cv:
    outer_folds: 3
    inner_folds: 3
    repeats: 1
    seed: 42
models:
  candidates:
  - logistic_regression
  - svm
  - random_forest
  selection_metric: f1
  selection_direction: max
imbalance:
  smote:
    enabled: false
    compare: false
metrics:
  primary:
  - f1
  - balanced_accuracy
  averaging: macro
  include_confidence

## 5. Run 1 — Technical Validation (Nested CV)


In [5]:
def run_cmd(cmd, cwd=None):
    print("$", " ".join(cmd))
    out = subprocess.run(cmd, cwd=cwd, capture_output=True, text=True)
    if out.stdout:
        print(out.stdout)
    if out.returncode != 0:
        if out.stderr:
            print(out.stderr)
        raise RuntimeError(f"Command failed ({out.returncode}): {' '.join(cmd)}")
    return out

def file_sha256(path: Path) -> str:
    h = hashlib.sha256()
    with path.open("rb") as f:
        for chunk in iter(lambda: f.read(1024 * 1024), b""):
            h.update(chunk)
    return h.hexdigest()

print("Training dataset hash:", file_sha256(TRAIN_CSV))

run_cmd([
    "classiflow", "project", "register-dataset", str(PROJECT_DIR),
    "--type", "train", "--manifest", str(TRAIN_CSV)
])

run_cmd([
    "classiflow", "project", "run-technical", str(PROJECT_DIR),
    "--run-id", "technical"
])

TECHNICAL_RUN = PROJECT_DIR / "runs" / "technical_validation" / "technical"
if not TECHNICAL_RUN.exists():
    raise FileNotFoundError(f"Technical run directory not found: {TECHNICAL_RUN}")

TECHNICAL_ALIAS = OUTPUT_ROOT / "technical"
if TECHNICAL_ALIAS.exists() and TECHNICAL_ALIAS.is_dir():
    shutil.rmtree(TECHNICAL_ALIAS)
shutil.copytree(TECHNICAL_RUN, TECHNICAL_ALIAS)
print("Technical artifacts mirrored to:", TECHNICAL_ALIAS)


Training dataset hash: 32311d51283f3207a485d3c1975af6ed26e40be8ad23e5d1efa4ff37f10ce243
$ classiflow project register-dataset runs/mrs_mb_meta_sklearn_technical/project --type train --manifest ../data/MBmerged-z-scores_MLready_correction.csv
Registered train dataset: 32311d51283f...

$ classiflow project run-technical runs/mrs_mb_meta_sklearn_technical/project --run-id technical
runs/mrs_mb_meta_sklearn_technical/project/runs/technical_validation/technical

Technical artifacts mirrored to: runs/mrs_mb_meta_sklearn_technical/technical


In [6]:
def require_file(path: Path, context: str):
    if not path.exists():
        raise FileNotFoundError(f"{context}: expected file not found -> {path}")
    return path

def find_latest_run_dir(base_dir: Path) -> Path:
    if not base_dir.exists():
        raise FileNotFoundError(f"Run base directory missing: {base_dir}")
    candidates = [p for p in base_dir.iterdir() if p.is_dir()]
    if not candidates:
        raise FileNotFoundError(f"No run directories found under: {base_dir}")
    return sorted(candidates, key=lambda p: p.stat().st_mtime)[-1]

def load_metrics(run_dir: Path):
    summary_json = run_dir / "metrics_summary.json"
    if summary_json.exists():
        return json.loads(summary_json.read_text(encoding="utf-8"))
    alt_csv = run_dir / "metrics_outer_meta_eval.csv"
    if alt_csv.exists():
        return {"fallback_csv": True, "table": pd.read_csv(alt_csv)}
    raise FileNotFoundError(f"No technical metrics artifact found in {run_dir}")

def load_predictions(run_dir: Path):
    preds = sorted(run_dir.glob("fold*/binary_*/predictions_outer_test.csv"))
    if preds:
        return pd.read_csv(preds[0])
    return None

def load_config(run_dir: Path):
    resolved = run_dir / "config.resolved.yaml"
    if resolved.exists():
        return yaml.safe_load(resolved.read_text(encoding="utf-8"))
    run_manifest = run_dir / "run.json"
    if run_manifest.exists():
        return json.loads(run_manifest.read_text(encoding="utf-8"))
    raise FileNotFoundError(f"No resolved config/run manifest found in {run_dir}")

def load_gate_results(project_dir: Path):
    payload_path = project_dir / "promotion" / "technical_gate_research_exploratory.json"
    if payload_path.exists():
        return json.loads(payload_path.read_text(encoding="utf-8"))
    raise FileNotFoundError(
        f"Technical gate payload not found: {payload_path}. Run gate-evaluation cell first."
    )

def find_inner_cv_csv(run_dir: Path) -> Path:
    candidates = [
        run_dir / "inner_cv_results.csv",
        run_dir / "metrics_inner_cv.csv",
        run_dir / "metrics_inner_cv_splits.csv",
    ]
    for p in candidates:
        if p.exists():
            return p
    discovered = sorted(run_dir.glob("*inner*cv*.csv"))
    if discovered:
        return discovered[0]
    available_csv = [p.name for p in sorted(run_dir.glob("*.csv"))]
    raise FileNotFoundError(
        f"No inner-CV CSV found in {run_dir}. Available CSV files: {available_csv}"
    )

metrics_payload = load_metrics(TECHNICAL_ALIAS)
outer_csv = require_file(TECHNICAL_ALIAS / "metrics_outer_meta_eval.csv", "Technical outer metrics")
inner_csv = find_inner_cv_csv(TECHNICAL_ALIAS)

outer_df = pd.read_csv(outer_csv)
outer_val = outer_df[outer_df.get("phase", "val") == "val"].copy()
inner_df = pd.read_csv(inner_csv)

print(f"Using inner CV file: {inner_csv.name}")

print("Technical metrics summary:")
if isinstance(metrics_payload, dict) and "summary" in metrics_payload:
    display(pd.DataFrame([metrics_payload["summary"]]))
else:
    print("metrics_summary.json unavailable; showing fallback table head")
    display(outer_val.head())

print("Inner CV selection summary (top rows):")
cols_pref = ["fold", "task", "model_name", "sampler", "mean_test_f1_macro", "mean_test_score", "rank_test_f1"]
cols = [c for c in cols_pref if c in inner_df.columns]
if cols:
    display(inner_df[cols].head(30))
else:
    print("Expected selection columns not found; showing first columns from inner CV table.")
    display(inner_df.head(30))


Using inner CV file: metrics_inner_cv.csv
Technical metrics summary:


,f1_macro,balanced_accuracy,recall,specificity,ppv,npv,precision,brier_calibrated,ece_calibrated,log_loss_calibrated
0,0.991475,0.990741,0.990741,0.996032,0.993056,0.996528,0.993056,0.021838,0.213387,0.257075


Inner CV selection summary (top rows):


,fold,task,model_name,sampler,mean_test_score
0,1,G3_vs_Rest,LogisticRegression,none,0.746032
1,1,G3_vs_Rest,LogisticRegression,none,0.746032
2,1,G3_vs_Rest,LogisticRegression,none,0.708791
3,1,G3_vs_Rest,LogisticRegression,none,0.690273
4,1,G3_vs_Rest,SVM,none,0.746032
5,1,G3_vs_Rest,SVM,none,0.698413
6,1,G3_vs_Rest,SVM,none,0.696825
7,1,G3_vs_Rest,SVM,none,0.696825
8,1,G3_vs_Rest,RandomForest,none,0.612698
9,1,G3_vs_Rest,RandomForest,none,0.634921


In [7]:
plot_metrics = ["f1_macro", "balanced_accuracy", "sensitivity", "specificity", "mcc", "roc_auc_ovr_macro"]
present = [m for m in plot_metrics if m in outer_val.columns]

if present:
    fig, ax = plt.subplots(figsize=(12, 5), constrained_layout=True)
    x = np.arange(len(outer_val))
    width = 0.12 if len(present) >= 5 else 0.18
    for i, m in enumerate(present):
        ax.bar(x + i * width, pd.to_numeric(outer_val[m], errors="coerce"), width=width, label=m)
    ax.set_title("Technical Validation Outer-Fold Metrics (Meta)")
    ax.set_xlabel("Outer Fold Row Index")
    ax.set_ylabel("Score")
    ax.set_ylim(0, 1.05)
    ax.set_xticks(x + width * (len(present)-1) / 2)
    ax.set_xticklabels([str(i + 1) for i in range(len(outer_val))])
    ax.legend()
    plt.show()
else:
    print("No expected key metrics found for plotting.")

cm_pngs = sorted(TECHNICAL_ALIAS.glob("fold*/binary_*/confusion_meta_fold*.png"))
if cm_pngs:
    plt.figure(figsize=(6, 6))
    plt.imshow(plt.imread(cm_pngs[0]))
    plt.axis("off")
    plt.title(f"Technical Confusion Matrix (example: {cm_pngs[0].name})")
    plt.show()
else:
    print("No saved confusion matrix image found in fold artifacts.")

roc_avg = TECHNICAL_ALIAS / "roc_meta_averaged.png"
if roc_avg.exists():
    plt.figure(figsize=(6, 5))
    plt.imshow(plt.imread(roc_avg))
    plt.axis("off")
    plt.title("Technical ROC (Averaged)")
    plt.show()
else:
    print("Averaged ROC plot not found.")

pr_avg = TECHNICAL_ALIAS / "pr_meta_averaged.png"
if pr_avg.exists():
    plt.figure(figsize=(6, 5))
    plt.imshow(plt.imread(pr_avg))
    plt.axis("off")
    plt.title("Technical PR (Averaged)")
    plt.show()
else:
    print("Averaged PR plot not found.")


/var/folders/qw/wt3crg911d32djy9xk8j0q8r0000gn/T/ipykernel_74743/1991860611.py:17: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


No saved confusion matrix image found in fold artifacts.


/var/folders/qw/wt3crg911d32djy9xk8j0q8r0000gn/T/ipykernel_74743/1991860611.py:37: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/var/folders/qw/wt3crg911d32djy9xk8j0q8r0000gn/T/ipykernel_74743/1991860611.py:47: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## 6. Promotion Gate Evaluation — Technical (Research / Exploratory)

This gate is computed against technical summary metrics only because no independent test set is available.

**Research / Exploratory Gate (layman explanation):**
The model shows promise and performs better than chance, but it is not yet considered clinically deployment-ready without independent-test confirmation.


In [8]:
template = get_promotion_gate_template("research_exploratory")
print("Gate template:", template.display_name)
print("Description:", template.description)
print("Layman:", template.layman_explanation)

summary = metrics_payload.get("summary", {}) if isinstance(metrics_payload, dict) else {}

# Fallback from outer val means if summary lacks fields
for col in ["f1_macro", "f1_weighted", "balanced_accuracy", "recall", "sensitivity", "mcc"]:
    if col not in summary and col in outer_val.columns:
        summary[col] = float(pd.to_numeric(outer_val[col], errors="coerce").mean())

rows = []
for gate in template.gates:
    observed = resolve_metric(summary, gate.metric)
    passed = (observed is not None) and (
        observed >= gate.threshold if gate.op == ">=" else
        observed > gate.threshold if gate.op == ">" else
        observed <= gate.threshold if gate.op == "<=" else
        observed < gate.threshold
    )
    rows.append({
        "metric": gate.metric,
        "op": gate.op,
        "threshold": gate.threshold,
        "observed_value": None if observed is None else float(observed),
        "passed": bool(passed),
        "aggregation": "mean",
    })

gate_table = pd.DataFrame(rows)
display(gate_table)

overall_pass = bool(gate_table["passed"].all()) if not gate_table.empty else False
print("Technical gate overall pass:", overall_pass)
print("Independent-test gate status: NOT EVALUATED (no independent test dataset configured)")

# Persist notebook-computed technical gate artifact
payload = {
    "template_id": template.template_id,
    "template_name": template.display_name,
    "layman_explanation": template.layman_explanation,
    "technical_only": True,
    "independent_test_evaluated": False,
    "technical_summary_metrics": summary,
    "per_gate_results": rows,
    "technical_overall_pass": overall_pass,
}
out_path = PROJECT_DIR / "promotion" / "technical_gate_research_exploratory.json"
out_path.write_text(json.dumps(payload, indent=2), encoding="utf-8")
print("Saved technical gate payload:", out_path)


Gate template: Research / Exploratory Gate
Description: Lenient baseline gate for exploratory model development and research triage.
Layman: The model shows promise and performs better than chance, but isn't ready for clinical use yet.


,metric,op,threshold,observed_value,passed,aggregation
0,Balanced Accuracy,>=,0.70,0.990741,True,mean
1,F1 Score,>=,0.65,0.991475,True,mean


Technical gate overall pass: True
Independent-test gate status: NOT EVALUATED (no independent test dataset configured)
Saved technical gate payload: runs/mrs_mb_meta_sklearn_technical/project/promotion/technical_gate_research_exploratory.json


## 7. Artifact Review / Provenance


In [9]:
def summarize_loaded(path: Path):
    try:
        if path.suffix in {".json", ".yaml", ".yml"}:
            text = path.read_text(encoding="utf-8")
            data = json.loads(text) if path.suffix == ".json" else yaml.safe_load(text)
            if isinstance(data, dict):
                return ", ".join(list(data.keys())[:8])
            return f"type={type(data).__name__}"
        if path.suffix == ".csv":
            df = pd.read_csv(path, nrows=5)
            return f"rows>= {len(df)}, cols={len(df.columns)}"
        return "binary/non-tabular"
    except Exception as exc:
        return f"load error: {exc}"

def pick_existing(*paths: Path):
    for p in paths:
        if p.exists():
            return p
    return None

# Meta runs may use metrics_inner_cv.csv instead of inner_cv_results.csv
inner_cv_artifact = pick_existing(
    TECHNICAL_ALIAS / "inner_cv_results.csv",
    TECHNICAL_ALIAS / "metrics_inner_cv.csv",
    TECHNICAL_ALIAS / "metrics_inner_cv_splits.csv",
)

# Optional helper for discovery visibility
available_csv = [p.name for p in sorted(TECHNICAL_ALIAS.glob("*.csv"))]

artifacts = [
    ("project_config", PROJECT_DIR / "project.yaml", "Project configuration YAML"),
    ("thresholds", PROJECT_DIR / "registry" / "thresholds.yaml", "Promotion gate template config"),
    ("technical_run_manifest", TECHNICAL_ALIAS / "run.json", "Training manifest for technical run"),
    ("technical_lineage", TECHNICAL_ALIAS / "lineage.json", "Run provenance and hashes"),
    ("technical_metrics_summary", TECHNICAL_ALIAS / "metrics_summary.json", "Aggregated technical metrics"),
    ("technical_outer_metrics", TECHNICAL_ALIAS / "metrics_outer_meta_eval.csv", "Per-fold outer validation metrics"),
    (
        "inner_cv_results",
        inner_cv_artifact if inner_cv_artifact is not None else (TECHNICAL_ALIAS / "inner_cv_results.csv"),
        "Inner CV model-selection table (or closest available inner-CV artifact)",
    ),
    ("resolved_config", TECHNICAL_ALIAS / "config.resolved.yaml", "Resolved config used for run"),
    ("technical_gate_payload", PROJECT_DIR / "promotion" / "technical_gate_research_exploratory.json", "Technical-only gate evaluation artifact"),
]

rows = []
for name, path, desc in artifacts:
    rows.append({
        "artifact_name": name,
        "path": str(path),
        "description": desc,
        "exists": path.exists(),
        "loaded_summary": summarize_loaded(path) if path.exists() else "missing",
    })

display(pd.DataFrame(rows))

if inner_cv_artifact is None:
    print("No inner-CV artifact found. Available CSV files in technical dir:")
    print(available_csv)
else:
    print(f"Inner-CV artifact used: {inner_cv_artifact.name}")

print("Effective config (from technical artifact):")
cfg = load_config(TECHNICAL_ALIAS)
if isinstance(cfg, dict):
    print(yaml.safe_dump(cfg, sort_keys=False)[:5000])
else:
    print(type(cfg))


,artifact_name,path,description,exists,loaded_summary
0,project_config,runs/mrs_mb_meta_sklearn_technical/project/pro...,Project configuration YAML,True,"project, data, key_columns, task, validation, ..."
1,thresholds,runs/mrs_mb_meta_sklearn_technical/project/reg...,Promotion gate template config,True,"technical_validation, independent_test, promot..."
2,technical_run_manifest,runs/mrs_mb_meta_sklearn_technical/technical/r...,Training manifest for technical run,True,"run_id, timestamp, package_version, training_d..."
3,technical_lineage,runs/mrs_mb_meta_sklearn_technical/technical/l...,Run provenance and hashes,True,"phase, run_id, timestamp_local, timestamp_utc,..."
4,technical_metrics_summary,runs/mrs_mb_meta_sklearn_technical/technical/m...,Aggregated technical metrics,True,"summary, per_fold"
5,technical_outer_metrics,runs/mrs_mb_meta_sklearn_technical/technical/m...,Per-fold outer validation metrics,True,"rows>= 5, cols=42"
6,inner_cv_results,runs/mrs_mb_meta_sklearn_technical/technical/m...,Inner CV model-selection table (or closest ava...,True,"rows>= 5, cols=11"
7,resolved_config,runs/mrs_mb_meta_sklearn_technical/technical/c...,Resolved config used for run,True,"project, data, key_columns, task, validation, ..."
8,technical_gate_payload,runs/mrs_mb_meta_sklearn_technical/project/pro...,Technical-only gate evaluation artifact,True,"template_id, template_name, layman_explanation..."


Inner-CV artifact used: metrics_inner_cv.csv
Effective config (from technical artifact):
project:
  id: MRS_MB_META_TECH
  name: MRS MB Meta Technical
  description: Reviewer notebook technical-validation-only run
  owner: local
data:
  train:
    manifest: /Users/alex/Documents/project-MLSubtype/data/MBmerged-z-scores_MLready_correction.csv
key_columns:
  label: MOLECULAR
task:
  mode: meta
  patient_stratified: false
validation:
  nested_cv:
    outer_folds: 3
    inner_folds: 3
    repeats: 1
    seed: 42
models:
  candidates:
  - logistic_regression
  - svm
  - random_forest
  selection_metric: f1
  selection_direction: max
imbalance:
  smote:
    enabled: false
    compare: false
multiclass:
  group_stratify: true
  sklearn:
    logreg:
      solver: saga
      multi_class: auto
      penalty: l2
      max_iter: 5000
      tol: 0.001
      C: 1.0
      class_weight: balanced
      n_jobs: -1
metrics:
  primary:
  - f1
  - balanced_accuracy
  averaging: macro
  include_confidence_i

## 8. Conclusions for Reviewers

This notebook reproduced a full **technical validation** pipeline for MB molecular group classification from MR spectroscopy using Classiflow meta mode on sklearn. It produced nested-CV artifacts, fold-level and aggregate metrics, and provenance files (`run.json`, `lineage.json`, resolved config, and CV tables/plots).

Promotion was assessed with the **Research / Exploratory Gate** using technical metrics only. Because no independent test dataset is configured, this notebook explicitly reports technical status as exploratory evidence and does not claim independent-test confirmation.
